# Imports

In [1]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import urllib.request
from urllib.parse import urlparse

import warnings
warnings.simplefilter(action='ignore')

# Load Data

In [3]:
df_train = pd.read_parquet('../input/movie-posters-2/df_train-split_v1.gzip')
df_val = pd.read_parquet('../input/movie-posters-2/df_eval-split_v1.gzip')
df_test = pd.read_parquet('../input/movie-posters-2/df_test-split_v1.gzip')

def get_path_from_url(url):
    if url is None:
        return None    
    return url.split('/')[-1]

df_train['filename'] = df_train['poster_url'].apply(get_path_from_url)
df_val['filename'] = df_val['poster_url'].apply(get_path_from_url)
df_test['filename'] = df_test['poster_url'].apply(get_path_from_url)

display(df_train.head(2))
display(df_val.head(2))
display(df_test.head(2))

,adult,id,original_title,popularity,video,url,poster_url,x0_Action,x0_Adventure,x0_Animation,...,x0_Horror,x0_Music,x0_Mystery,x0_Romance,x0_Science Fiction,x0_TV Movie,x0_Thriller,x0_War,x0_Western,filename
14384,False,52826,Nothing Lasts Forever,5.658,False,https://www.themoviedb.org/movie/52826,https://www.themoviedb.org/t/p/w500//ph2L3Rp3X...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,ph2L3Rp3XbMzxuLTTQBxvtNgF13.jpg
15751,False,460059,Burn Out,32.045,False,https://www.themoviedb.org/movie/460059,https://www.themoviedb.org/t/p/w500//3LeFOvzjZ...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3LeFOvzjZuIC7cQiXDeSIy1ym7a.jpg


,adult,id,original_title,popularity,video,url,poster_url,x0_Action,x0_Adventure,x0_Animation,...,x0_Horror,x0_Music,x0_Mystery,x0_Romance,x0_Science Fiction,x0_TV Movie,x0_Thriller,x0_War,x0_Western,filename
15689,False,3132,Bad Company,11.528,False,https://www.themoviedb.org/movie/3132,https://www.themoviedb.org/t/p/w500//umu141mcf...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,umu141mcfIEhRLgyQp7TWlGJFW.jpg
5511,False,44386,Pretty as a Picture: The Art of David Lynch,4.905,False,https://www.themoviedb.org/movie/44386,https://www.themoviedb.org/t/p/w500//uBW9g3w4a...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,uBW9g3w4aUZPch2MgU0D7ZdLKjs.jpg


,adult,id,original_title,popularity,video,url,poster_url,x0_Action,x0_Adventure,x0_Animation,...,x0_Horror,x0_Music,x0_Mystery,x0_Romance,x0_Science Fiction,x0_TV Movie,x0_Thriller,x0_War,x0_Western,filename
10340,False,157268,Death Doll,0.982,False,https://www.themoviedb.org/movie/157268,https://www.themoviedb.org/t/p/w500//jUU4cnuBv...,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jUU4cnuBvFtTuSSWNRGZhwDerlO.jpg
3376,False,19419,Screwed,7.348,False,https://www.themoviedb.org/movie/19419,https://www.themoviedb.org/t/p/w500//m6FfMbH1g...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,m6FfMbH1ga1Kp5QI9Nm0ETOIG64.jpg


In [4]:
df_train.shape, df_val.shape, df_test.shape

((11764, 27), (2942, 27), (3677, 27))

# Create Image generators for train, validate and test

In [7]:
IMAGES_DIR = '../input/movie-posters-2/data'

label_cols = [col for col in df_train.columns if col.startswith('x0_')]

In [11]:
datagen = ImageDataGenerator()

train_generator = datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col=label_cols,
    batch_size=64,
    shuffle=True,
    class_mode="raw",    
    target_size=(299, 299),
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=df_val,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col=label_cols,
    batch_size=64,
    class_mode="raw",
    target_size=(299, 299)
)

test_generator = datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col=label_cols,
    batch_size=64,
    class_mode="raw",
    target_size=(299, 299),
    validate_filenames=True
)

Found 11764 validated image filenames.
Found 2942 validated image filenames.
Found 3677 validated image filenames.


# Create Random Model

In [16]:
m1 = keras.Sequential()
m1.add(
    layers.Lambda(lambda x: tf.random.uniform((1,19), minval=0.0, maxval=1.0), input_shape=(299,299,3))
    )
m1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
m1.evaluate(test_generator)